2.计算相似度：

1)相似度度量：

2)获得相似样本：得到每个历史样本与待测样本最相近的片段，选择最相近的30个历史样本

3)RUL估计：加权核密度估计 对最近的历史样本的相似度进行min-max归一化处理作为权重，加权得到样本的RUL

## 1 Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import math
from sklearn.metrics.pairwise import euclidean_distances,cosine_similarity,paired_distances
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde
from fastdtw import fastdtw
import tslearn.metrics #DTW
from scipy.spatial.distance import euclidean,correlation,cosine,cdist
from scipy.stats import mode

## 读取数据

In [2]:
processed_data_file = '1_processed_data'
os.path.exists(processed_data_file)

True

In [3]:
train_df_pca = pd.read_csv(processed_data_file+'/train_PCA_data.csv')
test_df_pca = pd.read_csv(processed_data_file+'/test_PCA_data.csv')

In [4]:
train_group = train_df_pca.groupby('id')
test_group = test_df_pca.groupby('id')

In [5]:
n_train_units = len(train_group)
n_test_units = len(test_group)
n_train_units

100

In [6]:
train_group.get_group(1)
selected_ftr_name = ['p1']

## 计算相似度

### 计算等长序列的DTW距离

In [18]:
#DTW
MAX_DIST=99
sim_num = 30
on_win_size = 30
on_win_stride = 10
his_win_size = on_win_size
his_win_stride = 1
scale = 1#.5
ftr_len = len(selected_ftr_name)

In [20]:
dtw_dist_matrix = np.ones((n_test_units,40,n_train_units,350))*MAX_DIST

In [21]:
res_file = '2_res'
if os.path.exists(res_file)==False:
    os.makedirs(res_file)
#dtw_dist_matrix = np.load(res_file+'dtw_dist_matrix.npy')

### Coarse: 计算DTW距离
对test片段从后往前依次遍历

In [25]:
N_on_segs = []
t_begin = time.time()
for on_id in range(n_test_units):
    t0 = time.time()
    on_unit = test_group.get_group(on_id+1).reset_index(drop=True)
    on_len = len(on_unit)
    #print('---',on_id+1,'---')
    n_segs = int((on_len - on_win_size)/on_win_stride+1)
    N_on_segs += [n_segs]
    #print(on_len,n_segs)
    if on_len<on_win_size:
        continue
    #print(on_len)
    #print(n_segs)
    for ion_seg,on_seg in enumerate(range(on_len-on_win_size,0,-on_win_stride)):
        on_array = on_unit[on_seg:on_seg+on_win_size][selected_ftr_name]
        #print(ion_seg,on_seg, on_seg+sim_win_size)
        #print('---')
        #print(on_seg,len(on_array))
        #print(on_seg)
        #on_array = on_unit[on_seg]
        #print(on_len-sim_win_size - ion_seg*win_stride,on_len-ion_seg*win_stride)
        #on_array = on_unit[on_len-sim_win_size - ion_seg*win_stride:on_len-ion_seg*win_stride][selected_ftr_name]
        for his_id in range(n_train_units):
            #break
            his_unit = train_group.get_group(his_id+1).reset_index(drop=True)
            his_len = len(his_unit)
            #print(his_len)
            #print(lb_dist_matrix[on_id,ion_seg,his_id])
            for ihis_seg,his_seg in enumerate(range(0,his_len-his_win_size,his_win_stride)):
                #print(on_id,ion_seg,on_seg, 'his:',his_id,ihis_seg,his_seg,his_seg+sim_win_size)
                his_array = his_unit[his_seg:his_seg+his_win_size][selected_ftr_name]
                #print(his_seg,len(his_array))fastdtw(on_array,his_array)[0]#
                dtw_dist_matrix[on_id,ion_seg,his_id,ihis_seg] = tslearn.metrics.dtw_path(on_array, his_array)[1]
    t1= time.time()
    print('unit_id:',on_id,'segs:',n_segs,'time:',t1-t0,'avg:',(t1-t0)/n_segs)
t_end = time.time()
print('tot_time:',t_end-t_begin)

unit_id: 0 segs: 1 time: 12.412834644317627 avg: 12.412834644317627
unit_id: 1 segs: 2 time: 24.834654092788696 avg: 12.417327046394348
unit_id: 2 segs: 10 time: 124.806321144104 avg: 12.480632114410401
unit_id: 3 segs: 8 time: 99.54207420349121 avg: 12.442759275436401
unit_id: 4 segs: 7 time: 87.16803359985352 avg: 12.452576228550502
unit_id: 5 segs: 8 time: 101.74233531951904 avg: 12.71779191493988
unit_id: 6 segs: 14 time: 161.15240836143494 avg: 11.510886311531067
unit_id: 7 segs: 14 time: 173.19787693023682 avg: 12.371276923588344
unit_id: 8 segs: 3 time: 37.21607542037964 avg: 12.405358473459879
unit_id: 9 segs: 17 time: 210.77964663505554 avg: 12.398802743238562
unit_id: 10 segs: 6 time: 74.73501443862915 avg: 12.455835739771524
unit_id: 11 segs: 19 time: 235.24201369285583 avg: 12.381158615413465
unit_id: 12 segs: 17 time: 210.6921944618225 avg: 12.393658497754265
unit_id: 13 segs: 2 time: 24.62649631500244 avg: 12.31324815750122
unit_id: 14 segs: 5 time: 61.55995583534241 avg:

In [26]:
(t_end-t_begin)/np.sum(N_on_segs)

12.383550775778685

In [32]:
(t_end-t_begin)/1058

12.465483531384026

In [27]:
np.save(res_file+'\dtw_dist_matrix_'+str(on_win_size)+'_'+str(his_win_size)+'_'+str(on_win_stride)+'.npy',dtw_dist_matrix)

In [31]:
np.max(N_on_segs)

28